### Лекция 2. Пространства имён, функции и ввод-вывод

<br />

##### Функции, объявление и определение (declaration / definition)

Определение (definition) функции - описание её "интерфейса" (сигнатуры, возвращаемого типа и квалификаторов) И реализации.

```c++
float abs(float x)
{
    if (x >= 0)
        return x;
    return -x;
}

[[nodiscard]] float min_value(const std::vector<float>& items) noexcept
{
    float rv = items.front();
    for (float x : items)
        rv = std::min(rv, x);
    return rv;
}
```

Замечание:
* для поиска минимума, конечно же, используйте `std::min_element`
* а для вычисления абсолютных величин функцию `abs` из `#include <cmath>`

Объявление (declaration) функции - описание её "интерфейса"

```c++
float abs(float x);

[[nodiscard]] float min_value(const std::vector<float>& items) noexcept;
```

Если не прописано явно, у функции в программе может быть несколько объявлений, но не больше одного определения.

__Вопрос__: что помещается в header-файл, что в cpp-файл?

<br />

##### Передача аргументов в функцию и возвращаемое значение

Передача по значению и по ссылке
(нарисовать происходящее в памяти)

```c++
std::vector<float> v = {1.f, 2.f, 3.f};
```

Передача по значению - создание копии аргумента

```c++
float min_value_1(std::vector<float> items);

float m = min_value_1(v);
```

Передача по ссылке - работа с аргументом

```c++
float min_value_2(std::vector<float>& items);

float m = min_value_2(v);
```

Запрет на модификацию

```c++
float min_value_3(const std::vector<float>& items);

float m = min_value_3(v);
```

Что здесь происходит?

```c++
float f(std::vector<float>* items);
float f(const std::vector<float>* items);
float f(std::vector<float>* const items);
```

<br />

Возращаемое значение

```c++
void say_hello(const std::string& name)
{
    std::cout << "hello, " << name;
}
```

```c++
std::vector<std::string> make_team()
{
    return { "Bifur", "Bofur", "Bombur", "Oin",
             "Gloin", "Fili", "Nori", "Dori",
             "Dwalin", "Ori", "Balin", "Kili" };
}
```

```c++
bool append_teamlead(std::vector<std::string>& team)
{
    if (in_Shire())
    {
        team.push_back("Thorin");
        return true;
    }
    return false;
}
```

Немного примеров посложнее

```c++
std::string* find_balin(const std::vector<std::string>& team)
{
    for (const auto& member : team)
        if (member == "Balin")
            return &member;
    return nullptr;
}

// usage 1
std::vector<std::string> team = make_team();
if (std::string* maybe_balin = find_balin(team))
    std::cout << *maybe_balin;

// usage 2
if (std::string* maybe_balin = find_balin({"Ori", "Gloin", "Balin"}))
    std::cout << *maybe_balin;  // OOOPS
```

Ещё пример:

```c++
const std::string& get_value_from_cache(Cache c,
                                        int key,
                                        const std::string& default_value)
{
    if (c.contains(key))
        return c[key];
    return default_value;
}
// TODO: в чём проблема этой функции?
//           (а) конкретно про производительность
//           (б) их как минимум две

// usage 1
const std::string default_value = "Bilbo";
std::cout << get_value_from_cache(cache, missing_key, default_value);  // OK
        
// usage 2
std::cout << get_value_from_cache(cache, missing_key, "Bilbo");  // OK
        
// usage 3
std::string value = get_value_from_cache(cache, missing_key, "Bilbo");  // OK
std::cout << value;
        
// usage 4
const std::string& value = get_value_from_cache(cache, missing_key, "Bilbo");
std::cout << value;  // OOOOPS
```

__Вопрос__: что будет напечатано программой ниже?

Показать пример на godbolt.org на clang 8.0.0 с разными оптимизациями, чтобы наглядно продемонстрировать ub

```c++
#include <iostream>
#include <string>

const std::string& f(const bool x,
                     const std::string& default_value1,
                     const std::string& default_value2)
{
    return x ? default_value1 : default_value2;
}

int main()
{
    std::cout << "hello world" << std::endl;

    const std::string& s = f(true, "123", "12345");
    std::cout << s << std::endl;
    return 0;
}
```

<br />

##### Порядок вычисления аргументов

__Вопрос__: что будет распечатано в примерах?

Пример 1:

```c++
int a() { std::cout << 'a'; return 1; }
int b() { std::cout << 'b'; return 2; }
int c() { std::cout << 'c'; return 3; }

int sum(int a, int b int c) {
    return a + b + c;
}

int main() {
    std::cout << sum(a(), b(), c()) << std::endl;
    return 0;
}
```

Пример 2:

```c++
int a() { std::cout << 'a'; return 1; }
int b() { std::cout << 'b'; return 2; }
int c() { std::cout << 'c'; return 3; }

int main() {
    std::cout << a() + b() * c() << std::endl;
    return 0;
}
```

Ответ:

__Порядок вычисления аргументов не определён!__

https://en.cppreference.com/w/cpp/language/eval_order

<br />

##### Значения аргументов по умолчанию

Можно задавать значения аргументов по умолчанию:

```c++
std::string convert_to_string(int value, int base = 10);
```

```c++
std::string join_strings(const std::vector<std::string>& strings,
                         const std::string& sep = "\n");
```

использование:

```c++
std::string s1 = convert_to_string(42);    // 42
std::string s2 = convert_to_string(42, 2); // 101010

std::string song = join_strings({"In the town where I was born",
                                 "Lived a man who sailed to sea",
                                 "And he told us of his life",
                                 "In the land of submarines"});
std::string sentence = join_strings({"Run", "Forest", "run"}, " ");
```

Но такие аргументы должны быть последними:

```c++
std::string join_strings(const std::vector<std::string>& strings,
                         const std::string& sep = "\n",
                         bool skip_empty_lines);  // compilation ERROR!
```

```c++
std::string join_strings(const std::vector<std::string>& strings,
                         const std::string& sep = "\n",
                         bool skip_empty_lines = false);  // OK
```

<br />

##### Перегрузка функции

https://en.cppreference.com/book/intro/function_overloading

Задача - реализовать конвертацию всего в строку. Желательно единообразно и чтобы: есть способ - компилируется и работает, нет способа - не компилируется.

```c++
std::string convert_to_string(int x);       // 1
std::string convert_to_string(unsigned x);  // 2
std::string convert_to_string(float x);     // 3

std::cout << convert_to_string(5);    // 1
std::cout << convert_to_string(5u);   // 2
std::cout << convert_to_string(5.f);  // 3
```

<br />

##### Пространства имён и name mangling при компиляции

https://en.wikipedia.org/wiki/Name_mangling

Пространства времён решают проблему, когда функция с одинаковым именем имеет две реализации в разных билиотеках

```c++
// library for json parsing

// return version of json parsing library
std::string get_version();
```

```c++
// library for image manipulation

// return version of image manipulation library
std::string get_version();
```

Такой код скомпилируется, но не слинкуется, т.к. для функции `get_version` нарушено ODR-правило (One Definition Rule)

```c++
// library for json parsing

namespace cool_json_parser
{
    // return version of json parsing library
    std::string get_version();
}
```

```c++
// library for image manipulation

namespace cool_image_processing
{
   // return version of image manipulation library
    std::string get_version(); 
}
```

Такой код слинкуется, т.к. работает механизм name mangling во время компиляции

Показать пример на godbolt.org, не забыть убрать галочку demangle

```c++
#include <iostream>
#include <string>

std::string get_version(int x, int y, long z);

namespace cool_json_parser
{
    std::string get_version(int x, int y, long z);
}

namespace cool_image_processing
{
    std::string get_version(int x, int y, long z);
}

int main()
{
    std::cout << get_version(0, 0, 0);
    std::cout << cool_json_parser::get_version(0, 0, 0);
    std::cout << cool_image_processing::get_version(0, 0, 0);
    return 0;
}
```

В результате name mangling компилятор clang создаёт символы:
* `_Z11get_versionB5cxx11iil`
* `_ZN16cool_json_parser11get_versionB5cxx11Eiil`
* `_ZN21cool_image_processing11get_versionB5cxx11Eiil`

Какой из них к чему относится, догадаться не сложно. 

<br />

##### ADL (Argument Dependent Lookup)

https://en.cppreference.com/w/cpp/language/adl

Теперь мы реализовали свой класс `Point` в нашем `namespace math` и хотим, чтобы его объекты тоже можно было конвертировать в строку. Желательно единообразно.

```c++
namespace mymath
{
    struct Point
    {
        float x;
        float y;
    }
    
    std::string convert_to_string(Point p)
    {
        return convert_to_string(p.x) + " " + convert_to_string(p.y);
    }
}

int main()
{
    mymath::Point p{3.f, 4.f};
    
    // явно не указано пространство имён mymath у функции
    // convert_to_string, но ADL нужную функцию найдёт 
    std::cout << convert_to_string(p) << std::endl;
    
    return 0;
}
```

ADL - набор правил, который добавляет к области поиска функции пространства имён, содержащих её аргументы

<br />

##### Функции (операторы) ввода-вывода

Операторы - те же функции, но в альтернативной записи:

```c++
namespace std
{
    // есть в стандартной библиотеке
    string operator + (const string& l, const string& r);
}

namespace mymath
{
    // личный оператор сложения для точек
    Point operator + (const Point& l, const Point& r);
}

int main()
{
    std::string s1 = "s1", s2 = "s2";
    mymath::Point p1{0, 1}, p2{1, 0};
    
    std::string s = s1 + s2;
    mymath::Point p = p1 + p2;
    
    return 0;
}
```

Можно доопределить свои операторы для стандартных типов

```c++
namespace std
{
    string operator * (const string& l, unsigned count)
    {
        string rv;
        for (unsigned i = 0; i < count; ++i)
            rv += l;
        return rv;
    }
}

std::cout << std::string("nom ") * 3;  // "nom nom nom "
```

И их использование строится на тех же правилах: перегрузка + ADL

Пример для операторов вывода:

```c++
// from standard lib
namespace std
{
    std::ostream& operator << (std::ostream& os, int x);
    std::ostream& operator << (std::ostream& os, float x);
    std::ostream& operator << (std::ostream& os, const char* x);
    std::ostream& operator << (std::ostream& os, const std::string& x);
}

// your own
namespace mymath
{
    std::ostream& operator << (std::ostream& os, Point p)
    {
        return os << p.x << " " << p.y;
    }
}
```

<br />

__Замечания по лекции:__
* Про namespace - дорассказать пораньше. ADL, `nodiscard`, ввод-вывод вынести либо на конец курса либо во вторую часть курса.
* Чтобы понимали лучше рассказ про референсы на протухшие объекты, нужно рисовать происходящее в стеке и куче. Возможно, пример с `std::initializer_list` заменить на передачу `make_team` аргументом. Или сначала передачу `make_team`, а потом `std::initializer_list` для закрепления.
* Референсы на протухшие объекты - обязательно в повторение на следующую лекцию.
* Ещё раз повторить про указатели, ссылки и значения. Попробовать найти другое объяснение.